In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math
import folium
import csv
import json
import datetime
from ripple_helper import get_gateways_topology, get_gateways_df

# Processing transactions data

### Process transactions

In [2]:
#data = pd.read_csv('txs-parsed-jan-2013-aug-2017/more-txs-from-missing-parsed.txt', delimiter='\n', header=None)
data = pd.read_csv('txs-parsed-jan-2013-aug-2017/final_txs_2017.txt', delimiter='\n', header=None)

json_list = '['
for i in range(len(data)):
    json_text = data[0][i]
    json_list += json_text + ','
json_list = json_list[:-1]+']'

final_json = json.loads(json_list)
transactions = pd.DataFrame.from_dict(final_json)

## Get Exchange rates

In [3]:
rates = pd.read_csv('exchange_rate.csv', delimiter=',')

### Process gateways

In [4]:
gateways_df = pd.read_csv('gateways.csv', sep=',').set_index('Unnamed: 0')
gateways_df['asn'] = gateways_df['asn'].apply(lambda x: x[2:-2])
gateways_accounts = gateways_df.index

In [5]:
cols = ['actualIssuerReceiver','issuerReceiver','issuerSender','receiver','sender']
for c in cols:
    mask = (transactions[c] != '--') & (transactions[c] != 'nan')
    l = list(transactions[mask][c].values)
    count = 0
    for acc in l:
        if(acc in gateways_accounts):
                count += 1
    print('Column: {} - Percentage of gateways: {}'.format(c,count/len(l)))

Column: actualIssuerReceiver - Percentage of gateways: 0.2413851587544084
Column: issuerReceiver - Percentage of gateways: 0.26485471367175434
Column: issuerSender - Percentage of gateways: 0.09547136892453775
Column: receiver - Percentage of gateways: 0.024353950745963724
Column: sender - Percentage of gateways: 8.854902771683505e-05


In [6]:
account_to_asn = gateways_df['asn'].to_dict()
account_to_asn['rKiCet8SdvWxPXnAgYarFUXMh1zCPz432Y'] = '38895'

In [7]:
## Both represent Amazon
def sanitize(x):
    if x == '16509 38895':
        return '38895'
    return x

In [8]:
gateways_df['lat-lon'] = list(zip(gateways_df.latitude, gateways_df.longitude))
ases = gateways_df[['asn','lat-lon']].copy()
ases['asn'] = ases['asn'].apply(lambda x: sanitize(x))
ases = ases.set_index('asn').to_dict()['lat-lon']

In [9]:
links = []
with open('gateway_links.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        links.append(tuple([row[0],row[1],1]))

In [10]:
def resolve_sender_receiver(path):
    sender = ''
    for node in path:
        if node in gateways_accounts:
            sender = node
            break
    
    receiver = ''
    for node in reversed(path):
        if node in gateways_accounts:
            receiver = node
            
    if sender != receiver:
        return (sender,receiver)
    else:
        return None

In [11]:
def convert_date(d):
    readable = datetime.datetime.fromtimestamp(d + 946684800).isoformat()
    return readable[:7]

In [12]:
transactions_processed = pd.DataFrame(columns=['sender', 'receiver', 'amount', 'date'])

In [13]:
sender_cols = ['issuerSender', 'sender']
receiver_cols = ['issuerReceiver','receiver','actualIssuerReceiver']

for s in sender_cols:
    for r in receiver_cols:
        mask = transactions[s].isin(gateways_accounts) & transactions[r].isin(gateways_accounts)
        valid_transactions = transactions[mask][[s, r,'actualReceiverAmount', 'actualReceiverCurrency', 'date']]
        for index, row in valid_transactions.iterrows():
            source = account_to_asn[row[s]]
            dest = account_to_asn[row[r]]
            amount = float(row['actualReceiverAmount'])
            curr = row['actualReceiverCurrency']
            if (type(row['date']) is str or type(row['date']) is int):
                if(type(row['date']) is int):
                    d = convert_date(row['date'])

                if(type(row['date']) is str):
                    d = row['date'][:7]
            if (source != dest and not math.isnan(amount)):
                is_xrp = True
                if (curr != 'XRP'):
                    is_xrp = False
                    exchange = rates[(rates['date'] == d) & (rates['from'] == curr)]['exchange'].values
                    if(len(exchange) > 0):
                        amount = amount*exchange[0]
                        is_xrp = True
                    else:
                        is_xrp = False
                
                if (is_xrp):
                    transactions_processed = transactions_processed.append({'sender' : source , 'receiver' : dest, 'amount' : amount, 'date' : d} , ignore_index=True)

In [17]:
# For file more-txs-from-missing-parsed.txt
len(transactions_processed), len(transactions)

(6, 1599868)

In [14]:
# For file final_txs_2017.txt
len(transactions_processed), len(transactions)

(28116, 2021479)

In [24]:
transactions_processed.to_csv('transactions_processed.csv', mode='a')

## Special case for jan 2013 - dec 2016

The dataset does not have the same format, it would be too easy I guess

In [15]:
data = pd.read_csv('txs-parsed-jan-2013-aug-2017/ripple-transactions-jan-2013-dec-2016.txt', header=None)
transactions = data[[1,2,3,4,10]]
transactions.columns = ['sender','receiver','currency','amount','date']
transactions = transactions[transactions['amount'] != 0]

In [16]:
def convert_date(d):
    readable = datetime.datetime.fromtimestamp(d).isoformat()
    return readable[:7]

In [17]:
transactions['date'] = transactions['date'].apply(lambda x: convert_date(x))

In [18]:
transactions_processed = pd.DataFrame(columns=['sender', 'receiver', 'amount', 'date'])

In [19]:
mask = transactions['sender'].isin(gateways_accounts) & transactions['receiver'].isin(gateways_accounts)
valid_transactions = transactions[mask]
for index, row in valid_transactions.iterrows():
    source = account_to_asn[row['sender']]
    dest = account_to_asn[row['receiver']]
    amount = float(row['amount'])
    curr = row['currency']
    if (source != dest and amount > 0):
        is_xrp = True
        if (curr != 'XRP'):
            is_xrp = False
            exchange = rates[(rates['date'] == d) & (rates['from'] == curr)]['exchange'].values
            if(len(exchange) > 0):
                amount = amount*exchange[0]
                is_xrp = True
            else:
                is_xrp = False

        if (is_xrp):
            transactions_processed = transactions_processed.append({'sender' : source , 'receiver' : dest, 'amount' : amount, 'date' : d} , ignore_index=True)

In [20]:
# For file ripple-transactions-jan-2013-dec-2016.txt
len(transactions_processed), len(transactions)

(11, 20680411)

Due to the number of usable transactions for each file, I decided to keep only the transactions of 2017